In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
import pandas as pd

df = pd.read_excel("../data/raw/emdat.xlsx")
df.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,GADM Admin Units,Entry Date,Last Update
0,2010-0416-IND,No,nat-hyd-flo-riv,Natural,Hydrological,Flood,Riverine flood,NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,69.513293,"[{""adm2_code"":70091,""adm2_name"":""Lakhimpur""}]","[{""gid_2"":""IND.4.20_1"",""migration_date"":""2025-...",2011-01-10,2025-12-20
1,2023-0846-IND,No,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,97.134993,NaN,"[{""gid_2"":""IND.31.25_1"",""name_2"":""Thoothukkudi...",2023-12-21,2026-01-16
2,2018-0049-IND,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,80.049596,"[{""adm2_code"":17633,""adm2_name"":""Delhi""}]","[{""gid_2"":""IND.25.1_1"",""migration_date"":""2025-...",2018-02-27,2025-12-20
3,2000-0444-IND,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,54.895152,"[{""adm2_code"":70041,""adm2_name"":""Administrativ...","[{""gid_2"":""IND.3.5_1"",""migration_date"":""2025-1...",2003-07-01,2025-12-20
4,2000-0332-IND,No,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,IND,...,NaN,NaN,NaN,NaN,NaN,54.895152,"[{""adm2_code"":17582,""adm2_name"":""Golaghat""},{""...","[{""gid_2"":""IND.3.11_1"",""migration_date"":""2025-...",2005-09-15,2025-12-20


In [3]:
df.shape
df.info()
df.columns

<class 'pandas.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 47 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   DisNo.                                     392 non-null    str    
 1   Historic                                   392 non-null    str    
 2   Classification Key                         392 non-null    str    
 3   Disaster Group                             392 non-null    str    
 4   Disaster Subgroup                          392 non-null    str    
 5   Disaster Type                              392 non-null    str    
 6   Disaster Subtype                           392 non-null    str    
 7   External IDs                               105 non-null    str    
 8   Event Name                                 36 non-null     str    
 9   ISO                                        392 non-null    str    
 10  Country                              

Index(['DisNo.', 'Historic', 'Classification Key', 'Disaster Group',
       'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype',
       'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region',
       'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', 'AID Contribution ('000 US$)', 'Magnitude',
       'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', 'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'Total Damage ('000 US$)', 'Total Damage, Adjusted ('000 US$)', 'CPI',
       'Admin Units', 'GADM Admin Units', 'Entry Date', 'Last Update'],
      dtype='str')

In [4]:
core_cols = [
    "Disaster Type",
    "Country",
    "Start Year",
    "Total Deaths",
    "No. Injured",
    "No. Affected",
    "No. Homeless",
    "Total Affected",
    "Total Damage ('000 US$)"
]

core_df = df[core_cols]
core_df.head()

,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,NaN,NaN,30000.0,NaN,30000.0,NaN
1,Flood,India,2023,10.0,NaN,NaN,NaN,NaN,NaN
2,Extreme temperature,India,2018,44.0,NaN,NaN,NaN,NaN,NaN
3,Flood,India,2000,130.0,NaN,NaN,NaN,NaN,NaN
4,Flood,India,2000,20.0,NaN,3000000.0,NaN,3000000.0,NaN


In [5]:
core_df.isnull().sum

<bound method DataFrame.sum of      Disaster Type  Country  Start Year  Total Deaths  No. Injured  \
0            False    False       False          True         True   
1            False    False       False         False         True   
2            False    False       False         False         True   
3            False    False       False         False         True   
4            False    False       False         False         True   
..             ...      ...         ...           ...          ...   
387          False    False       False         False         True   
388          False    False       False         False         True   
389          False    False       False         False         True   
390          False    False       False         False         True   
391          False    False       False         False         True   

     No. Affected  No. Homeless  Total Affected  Total Damage ('000 US$)  
0           False          True           False      

In [6]:
from src.preprocess import load_and_clean_data

clean_df = load_and_clean_data("../data/raw/emdat.xlsx")
clean_df.info()
clean_df.head()

<class 'pandas.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Disaster Type            392 non-null    str    
 1   Country                  392 non-null    str    
 2   Start Year               392 non-null    int64  
 3   Total Deaths             392 non-null    float64
 4   No. Injured              392 non-null    float64
 5   No. Affected             392 non-null    float64
 6   No. Homeless             392 non-null    float64
 7   Total Affected           392 non-null    float64
 8   Total Damage ('000 US$)  392 non-null    float64
dtypes: float64(6), int64(1), str(2)
memory usage: 27.7 KB


,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,0.0,0.0,30000.0,0.0,30000.0,0.0
1,Flood,India,2023,10.0,0.0,0.0,0.0,0.0,0.0
2,Extreme temperature,India,2018,44.0,0.0,0.0,0.0,0.0,0.0
3,Flood,India,2000,130.0,0.0,0.0,0.0,0.0,0.0
4,Flood,India,2000,20.0,0.0,3000000.0,0.0,3000000.0,0.0


In [7]:
clean_df.head()

,Disaster Type,Country,Start Year,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Total Damage ('000 US$)
0,Flood,India,2010,0.0,0.0,30000.0,0.0,30000.0,0.0
1,Flood,India,2023,10.0,0.0,0.0,0.0,0.0,0.0
2,Extreme temperature,India,2018,44.0,0.0,0.0,0.0,0.0,0.0
3,Flood,India,2000,130.0,0.0,0.0,0.0,0.0,0.0
4,Flood,India,2000,20.0,0.0,3000000.0,0.0,3000000.0,0.0


In [8]:
from src.preprocess import load_and_clean_data
from src.feature_engineering import add_severity_score, add_severity_level

df = load_and_clean_data("../data/raw/emdat.xlsx")

df = add_severity_score(df)
df = add_severity_level(df)

df["severity_level"] = df["severity_level"].map({
    0: "Low",
    1: "Medium",
    2: "High"
})

df[["severity_score", "severity_level"]].head()

df["severity_level"].value_counts()

severity_level
High      133
Low       130
Medium    129
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df.drop(columns=["severity_level", "severity_score"])
y = df["severity_level"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [11]:
df.columns

Index(['Disaster Type', 'Country', 'Start Year', 'Total Deaths', 'No. Injured',
       'No. Affected', 'No. Homeless', 'Total Affected',
       'Total Damage ('000 US$)', 'log_deaths', 'log_injured', 'log_affected',
       'log_damage', 'severity_score', 'severity_level'],
      dtype='str')

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
X_train.dtypes


Disaster Type                  str
Country                        str
Start Year                   int64
Total Deaths               float64
No. Injured                float64
No. Affected               float64
No. Homeless               float64
Total Affected             float64
Total Damage ('000 US$)    float64
log_deaths                 float64
log_injured                float64
log_affected               float64
log_damage                 float64
dtype: object

In [14]:
X = pd.get_dummies(X, drop_first=True)


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.2,
    random_state = 42,
    stratify = y
)


In [16]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, y_train)


d:\disaster_relief_ml\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [17]:
y_pred = model.predict(X_test)


In [18]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        High       0.84      0.78      0.81        27
         Low       0.77      0.92      0.84        26
      Medium       0.65      0.58      0.61        26

    accuracy                           0.76        79
   macro avg       0.76      0.76      0.75        79
weighted avg       0.76      0.76      0.75        79



In [19]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)


array([[21,  0,  6],
       [ 0, 24,  2],
       [ 4,  7, 15]])

In [20]:
from sklearn.ensemble import RandomForestClassifier


In [21]:
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    class_weight="balanced"
)


In [22]:
rf_model.fit(X_train, y_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [23]:
y_pred_rf = rf_model.predict(X_test)


In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred_rf))


              precision    recall  f1-score   support

        High       0.96      0.93      0.94        27
         Low       1.00      0.96      0.98        26
      Medium       0.89      0.96      0.93        26

    accuracy                           0.95        79
   macro avg       0.95      0.95      0.95        79
weighted avg       0.95      0.95      0.95        79



In [26]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[21,  0,  6],
       [ 0, 24,  2],
       [ 4,  7, 15]])

In [27]:
from src.decision_engine import recommend_action

sample_prediction = model.predict(X_test.iloc[[0]])[0]
decision = recommend_action(sample_prediction)

sample_prediction, decision


('Medium',
 {'priority': 2, 'action': 'Deploy medical teams and emergency supplies'})

In [28]:
probs = model.predict_proba(X_test.iloc[[0]])
confidence = probs.max()

confidence


np.float64(0.5312499433747481)